In [10]:
from vllm import LLM, SamplingParams
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [2]:
model_path = 'deepseek-ai/DeepSeek-Prover-V1.5-RL'

In [3]:
llm = LLM(model=model_path, max_num_batched_tokens=8192, seed=0, trust_remote_code=True)

INFO 09-23 02:35:44 llm_engine.py:98] Initializing an LLM engine (v0.4.1) with config: model='deepseek-ai/DeepSeek-Prover-V1.5-RL', speculative_config=None, tokenizer='deepseek-ai/DeepSeek-Prover-V1.5-RL', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0)


/home/ubuntu/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 09-23 02:35:44 utils.py:608] Found nccl from library /home/ubuntu/.config/vllm/nccl/cu12/libnccl.so.2.18.1
INFO 09-23 02:35:44 selector.py:28] Using FlashAttention backend.
INFO 09-23 02:35:45 weight_utils.py:193] Using model weights format ['*.safetensors']
INFO 09-23 02:35:49 model_runner.py:173] Loading model weights took 12.8725 GB
INFO 09-23 02:35:50 gpu_executor.py:119] # GPU blocks: 2858, # CPU blocks: 546
INFO 09-23 02:35:56 model_runner.py:976] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 09-23 02:35:56 model_runner.py:980] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 09-23 02:35:59 model_runner.py:1057] Graph capturing fin

In [7]:
auto_llm = AutoModelForCausalLM.from_pretrained(
        'deepseek-ai/DeepSeek-Prover-V1.5-RL',
        trust_remote_code=True,
        device_map='auto',
        cache_dir='model_weights'
    )

/home/ubuntu/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model-00001-of-000002.safetensors:  29%|##9       | 2.50G/8.59G [00:00<?, ?B/s]

model-00002-of-000002.safetensors:   0%|          | 0.00/5.23G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device device because they were offloaded to the cpu.


In [ ]:
auto_llm

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(102400, 4096)
    (layers): ModuleList(
      (0-29): 30 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head)

In [16]:
llm.llm_engine.model_config

In [29]:
llm.

{'llm_engine': <vllm.engine.llm_engine.LLMEngine at 0x7f0cb0956380>,
 'request_counter': <vllm.utils.Counter at 0x7f0cac114f10>}

In [25]:
x = "Show x^2 > 0"

# Initialize the tokenizer with the same model path
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

# Tokenize the input text
inputs = tokenizer(x, return_tensors='pt')

# Ensure the model outputs hidden states
model = llm.generate
llm.llm_engine.model_config.output_hidden_states = True

# Pass the inputs through the model
with torch.no_grad():
    print("inputs", inputs)
    outputs = llm.generate(x)

print("outputs", outputs)
# Retrieve hidden states from the outputs
hidden_states = outputs.hidden_states

# Extract the activations from the 25th intermediate layer
# Note: hidden_states[0] is the embedding layer, so hidden_states[25] corresponds to the 25th layer
layer_num = 25
layer_25_activations = hidden_states[layer_num]

print("Activations of the 25th intermediate layer:")
print(layer_25_activations)


/home/ubuntu/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


inputs {'input_ids': tensor([[100000,  12423,   1376,     61,     17,   1879,    207,     15]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  4.75it/s]

outputs [RequestOutput(request_id=1, prompt='Show x^2 > 0', prompt_token_ids=[100000, 12423, 1376, 61, 17, 1879, 207, 15], prompt_logprobs=None, outputs=[CompletionOutput(index=0, text=' for all x in Re #11045\nTo show that', token_ids=[327, 521, 1376, 279, 1926, 1501, 16, 16, 15, 19, 20, 185, 100000, 1898, 1296, 344], cumulative_logprob=-22.468135497299954, logprobs=None, finish_reason=length, stop_reason=None)], finished=True, metrics=RequestMetrics(arrival_time=1727059882.28389, last_token_time=1727059882.28389, first_scheduled_time=1727059882.2856731, first_token_time=1727059882.300276, time_in_queue=0.001783132553100586, finished_time=1727059882.4949777), lora_request=None)]


AttributeError: 'list' object has no attribute 'hidden_states'

In [30]:
from vllm import ModelRegistry
ModelRegistry.register_model(model_path + "-clone", llm)

In [31]:
llm_clone = LLM(model=model_path + "-clone", max_num_batched_tokens=8192, seed=0, trust_remote_code=True)

/home/ubuntu/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


OSError: deepseek-ai/DeepSeek-Prover-V1.5-RL-clone is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

## Upload truncated model to HF

Truncate model

In [34]:
auto_llm = AutoModelForCausalLM.from_pretrained(
        'deepseek-ai/DeepSeek-Prover-V1.5-RL',
        trust_remote_code=True,
        device_map='auto',
        cache_dir='model_weights'
    )

/home/ubuntu/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [35]:
original_num_layers = auto_llm.config.num_hidden_layers
assert(original_num_layers == 30)
auto_llm.config.num_hidden_layers = original_num_layers - 5

30
